In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
df = pd.read_csv("/kaggle/input/playground-series-s4e2/train.csv")

In [3]:
df.describe()

,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,20758.00000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000
mean,10378.50000,23.841804,1.700245,87.887768,2.445908,2.761332,2.029418,0.981747,0.616756
std,5992.46278,5.688072,0.087312,26.379443,0.533218,0.705375,0.608467,0.838302,0.602113
min,0.00000,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,5189.25000,20.000000,1.631856,66.000000,2.000000,3.000000,1.792022,0.008013,0.000000
50%,10378.50000,22.815416,1.700000,84.064875,2.393837,3.000000,2.000000,1.000000,0.573887
75%,15567.75000,26.000000,1.762887,111.600553,3.000000,3.000000,2.549617,1.587406,1.000000
max,20757.00000,61.000000,1.975663,165.057269,3.000000,4.000000,3.000000,3.000000,2.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   Gender                          20758 non-null  object 
 2   Age                             20758 non-null  float64
 3   Height                          20758 non-null  float64
 4   Weight                          20758 non-null  float64
 5   family_history_with_overweight  20758 non-null  object 
 6   FAVC                            20758 non-null  object 
 7   FCVC                            20758 non-null  float64
 8   NCP                             20758 non-null  float64
 9   CAEC                            20758 non-null  object 
 10  SMOKE                           20758 non-null  object 
 11  CH2O                            20758 non-null  float64
 12  SCC                             

In [5]:
for a in df.columns:
    print(a,len(df.loc[:,a].unique()))

id 20758
Gender 2
Age 1703
Height 1833
Weight 1979
family_history_with_overweight 2
FAVC 2
FCVC 934
NCP 689
CAEC 4
SMOKE 2
CH2O 1506
SCC 2
FAF 1360
TUE 1297
CALC 3
MTRANS 5
NObeyesdad 7


In [6]:
df.sample(4)

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
11486,11486,Male,37.524551,1.700000,85.000000,yes,yes,2.000000,3.0,Sometimes,no,1.927976,no,1.327193,0.000000,no,Automobile,Overweight_Level_II
10206,10206,Male,29.713169,1.856315,113.234349,yes,yes,2.333503,3.0,Sometimes,no,2.003563,no,0.000000,1.898139,Sometimes,Automobile,Obesity_Type_II
13854,13854,Female,26.000000,1.632983,111.946321,yes,yes,3.000000,3.0,Sometimes,no,2.737353,no,0.000000,0.091711,Sometimes,Public_Transportation,Obesity_Type_III
3875,3875,Male,30.958051,1.726279,125.064264,yes,yes,2.499108,3.0,Sometimes,no,1.818026,no,0.627700,0.402075,Sometimes,Public_Transportation,Obesity_Type_II


In [7]:
df.NObeyesdad.value_counts()

NObeyesdad
Obesity_Type_III       4046
Obesity_Type_II        3248
Normal_Weight          3082
Obesity_Type_I         2910
Insufficient_Weight    2523
Overweight_Level_II    2522
Overweight_Level_I     2427
Name: count, dtype: int64

In [8]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [9]:
def preprocess(dff):
    categorical_columns = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
    dummies = pd.get_dummies(dff[categorical_columns]).astype(int)
    dff.drop(columns=categorical_columns, inplace=True)
    final = pd.concat([dff, dummies], axis=1)
    return final

In [10]:
x = preprocess(x)
x=x.iloc[:,1:]
x

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,...,SCC_no,SCC_yes,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,24.443011,1.699998,81.669950,2.000000,2.983297,2.763573,0.000000,0.976473,0,1,...,1,0,0,1,0,0,0,0,1,0
1,18.000000,1.560000,57.000000,2.000000,3.000000,2.000000,1.000000,1.000000,1,0,...,1,0,0,0,1,1,0,0,0,0
2,18.000000,1.711460,50.165754,1.880534,1.411685,1.910378,0.866045,1.673584,1,0,...,1,0,0,0,1,0,0,0,1,0
3,20.952737,1.710730,131.274851,3.000000,3.000000,1.674061,1.467863,0.780199,1,0,...,1,0,0,1,0,0,0,0,1,0
4,31.641081,1.914186,93.798055,2.679664,1.971472,1.979848,1.967973,0.931721,0,1,...,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,25.137087,1.766626,114.187096,2.919584,3.000000,2.151809,1.330519,0.196680,0,1,...,1,0,0,1,0,0,0,0,1,0
20754,18.000000,1.710000,50.000000,3.000000,4.000000,1.000000,2.000000,1.000000,0,1,...,1,0,0,1,0,0,0,0,1,0
20755,20.101026,1.819557,105.580491,2.407817,3.000000,2.000000,1.158040,1.198439,0,1,...,1,0,0,0,1,0,0,0,1,0
20756,33.852953,1.700000,83.520113,2.671238,1.971472,2.144838,0.000000,0.973834,0,1,...,1,0,0,0,1,1,0,0,0,0


In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
y

array([6, 1, 0, ..., 3, 6, 3])

In [12]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=2000, random_state=42)
clf.fit(x, y)

RandomForestClassifier(n_estimators=2000, random_state=42)

In [13]:
test = pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv")
test.loc[test['CALC'] == 'Always', 'CALC'] = 'Frequently'
test2 = preprocess(test)

In [14]:
test2 = test2.iloc[:,1:]
test2

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,...,SCC_no,SCC_yes,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,26.899886,1.848294,120.644178,2.938616,3.000000,2.825629,0.855400,0.000000,0,1,...,1,0,0,1,0,0,0,0,1,0
1,21.000000,1.600000,66.000000,2.000000,1.000000,3.000000,1.000000,0.000000,1,0,...,1,0,0,1,0,0,0,0,1,0
2,26.000000,1.643355,111.600553,3.000000,3.000000,2.621877,0.000000,0.250502,1,0,...,1,0,0,1,0,0,0,0,1,0
3,20.979254,1.553127,103.669116,2.000000,2.977909,2.786417,0.094851,0.000000,0,1,...,1,0,0,1,0,0,0,0,1,0
4,26.000000,1.627396,104.835346,3.000000,3.000000,2.653531,0.000000,0.741069,1,0,...,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,23.327836,1.721384,78.030383,2.813234,3.000000,1.000000,0.807076,0.778632,0,1,...,1,0,0,1,0,0,0,0,1,0
13836,29.000000,1.590000,62.000000,3.000000,3.000000,2.000000,0.000000,0.000000,1,0,...,1,0,0,1,0,0,0,0,1,0
13837,22.935612,1.585547,44.376637,3.000000,2.273740,2.000000,1.949840,1.000000,1,0,...,1,0,0,1,0,0,0,0,1,0
13838,21.000000,1.620000,53.000000,2.000000,3.000000,2.000000,3.000000,2.000000,0,1,...,1,0,0,0,1,0,0,0,1,0


In [15]:
test2.reindex(columns=x.columns, fill_value=0)

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,...,SCC_no,SCC_yes,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,26.899886,1.848294,120.644178,2.938616,3.000000,2.825629,0.855400,0.000000,0,1,...,1,0,0,1,0,0,0,0,1,0
1,21.000000,1.600000,66.000000,2.000000,1.000000,3.000000,1.000000,0.000000,1,0,...,1,0,0,1,0,0,0,0,1,0
2,26.000000,1.643355,111.600553,3.000000,3.000000,2.621877,0.000000,0.250502,1,0,...,1,0,0,1,0,0,0,0,1,0
3,20.979254,1.553127,103.669116,2.000000,2.977909,2.786417,0.094851,0.000000,0,1,...,1,0,0,1,0,0,0,0,1,0
4,26.000000,1.627396,104.835346,3.000000,3.000000,2.653531,0.000000,0.741069,1,0,...,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,23.327836,1.721384,78.030383,2.813234,3.000000,1.000000,0.807076,0.778632,0,1,...,1,0,0,1,0,0,0,0,1,0
13836,29.000000,1.590000,62.000000,3.000000,3.000000,2.000000,0.000000,0.000000,1,0,...,1,0,0,1,0,0,0,0,1,0
13837,22.935612,1.585547,44.376637,3.000000,2.273740,2.000000,1.949840,1.000000,1,0,...,1,0,0,1,0,0,0,0,1,0
13838,21.000000,1.620000,53.000000,2.000000,3.000000,2.000000,3.000000,2.000000,0,1,...,1,0,0,0,1,0,0,0,1,0


In [16]:
predictions = clf.predict(test2)
actual_labels = le.inverse_transform(predictions)

In [17]:
test = pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv")
sub = pd.DataFrame({"id":test.id,"NObeyesdad":actual_labels})

In [18]:
sub.to_csv("submission2.csv",index=False)